In [2]:
import numpy as np
def build_adjM(verNum:int, edges:list[tuple[int, int, float]], is_directed=False):
    """构建邻接矩阵, 顶点编号从1开始"""

    adjM=np.full(shape=(verNum+1, verNum+1), fill_value=np.inf)
    
    # 1<= u, v <= verNum
    if is_directed:
        for u, v, w in edges:
            adjM[u][v]=w
    
    else:
        for u, v, w in edges:
            adjM[u][v]=adjM[v][u]=w

    return adjM


def floyd(adjM:np.ndarray)->list[list[list[int]]]:
    """
    Floyd-Warshall算法求解所有节点对的最短路径
    :param adjM: 邻接矩阵（顶点编号从1开始）
    
    :return: 返回距离矩阵, 任意两节点之间的最短路径


    Floyd 算法适用范围:
        可以有负边权, 但不可以有负环.

    算法大意:
        动态规划：逐步允许更多中间点参与，更新任意两点间最短距离.
        详见视频 https://www.bilibili.com/video/BV1aN411d7Yi?spm_id_from=333.788.videopod.episodes&vd_source=c8e4e809f91f46885a44be8339a7976c&p=63
        或算法课 PPT

    复杂度分析:
        初始化部分(构建邻接矩阵 adjM, 给 dist, rec赋值等)复杂度 O(|V|^2).
        计算递推式部分, 复杂度 O(|V|^3).

        因此, 总的复杂度 O(|V|^3).
    """
    verNum=adjM.shape[0]-1

    dist=np.copy(adjM) # 深拷贝距离矩阵，避免修改原邻接矩阵
    for u in range(1, verNum+1):
        dist[u][u]=0

    next = [[0]*(verNum+1) for _ in range(verNum+1)]
    # next: 方便存储并且找到任意两点之间最短路径而设置的辅助数组
    # next[i][j] = 从 i 出发，走向 j 的最短路径上，下一个点是谁
    for u, v, w in edges:
        next[u][v] = v
        next[v][u] = u   # 无向图
    
    for k in range(1, verNum+1):
        for i in range(1, verNum+1):
            for j in range(1, verNum+1):
                if dist[i][k] != np.inf and dist[k][j] != np.inf:
                    tmp=dist[i][k]+dist[k][j]
                    if tmp<dist[i][j]:
                        dist[i][j]=tmp
                        next[i][j]=next[i][k]
    
    def global_trace_back_shortest_path():
        
        def get_path(u, v):
            if next[u][v] == 0:
                return []
            path = [u]
            while u != v:
                u = next[u][v]
                path.append(u)
            return path

        shortest_paths=[[[] for _ in range(verNum)] for _ in range(verNum)]
        for i in range(1, verNum+1):
            for j in range(1, verNum+1):
                shortest_paths[i-1][j-1]=get_path(i, j)
        return shortest_paths

    for i in range(1, verNum+1):
        print(next[i][1:])
    return global_trace_back_shortest_path()

![Test Case 1](TestCase1.png)

In [ ]:
'''
Test Case1: 如上图, 出处:大一下<数据结构>PPT
下分别计算从下标为i 的结点到其余节点的最短距离

Floyd 的代码和这里对不上, 因为下标一个是 0_base, 另一个是 1_base, 要修改
'''
vertexNum=7
edges=[(0, 1, 10), (0, 2, 2), (1, 4, 1), (2, 3, 2), (3, 4, 4),
       (3, 5, 6), (2, 5, 11), (4, 6, 7), (5, 6, 3)]

adjM=build_adjM(vertexNum, edges, is_directed=False)
shortest_paths:list[list[list[int]]]=floyd(adjM)
for i in range(vertexNum):
    for j in range(vertexNum):
        print(f'{i+1}->{j+1}: {shortest_paths[i][j]}')
        

[0, 4, 4, 4, 4, 4, 0]
[3, 0, 3, 3, 3, 3, 0]
[4, 2, 0, 4, 5, 5, 0]
[1, 3, 3, 0, 3, 6, 0]
[3, 3, 3, 3, 0, 6, 0]
[4, 5, 5, 4, 5, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
1->1: []
1->2: [1, 4, 3, 2]
1->3: [1, 4, 3]
1->4: [1, 4]
1->5: [1, 4, 3, 5]
1->6: [1, 4, 6]
1->7: []
2->1: [2, 3, 4, 1]
2->2: []
2->3: [2, 3]
2->4: [2, 3, 4]
2->5: [2, 3, 5]
2->6: [2, 3, 5, 6]
2->7: []
3->1: [3, 4, 1]
3->2: [3, 2]
3->3: []
3->4: [3, 4]
3->5: [3, 5]
3->6: [3, 5, 6]
3->7: []
4->1: [4, 1]
4->2: [4, 3, 2]
4->3: [4, 3]
4->4: []
4->5: [4, 3, 5]
4->6: [4, 6]
4->7: []
5->1: [5, 3, 4, 1]
5->2: [5, 3, 2]
5->3: [5, 3]
5->4: [5, 3, 4]
5->5: []
5->6: [5, 6]
5->7: []
6->1: [6, 4, 1]
6->2: [6, 5, 3, 2]
6->3: [6, 5, 3]
6->4: [6, 4]
6->5: [6, 5]
6->6: []
6->7: []
7->1: []
7->2: []
7->3: []
7->4: []
7->5: []
7->6: []
7->7: []


![Test Case 2](TestCase2.png)

In [6]:
'''
Test Case2: 如上图, 出处:大一下<数据结构>PPT
下分别计算从下标为i 的结点到其余节点的最短距离

Floyd 的代码和这里对不上, 因为下标一个是 0_base, 另一个是 1_base, 要修改
'''
vertexNum=7
edges=[(0, 1, 10), (0, 2, 2), (1, 4, 1), (2, 3, 2), (3, 4, 4),
       (3, 5, 6), (2, 5, 11), (4, 6, 7), (5, 6, 3)]

adjM=build_adjM(vertexNum, edges, is_directed=False)

shortest_paths:list[list[list[int]]]=floyd(adjM)
for i in range(vertexNum):
    for j in range(vertexNum):
        print(f'{i+1}->{j+1}: {shortest_paths[i][j]}')    

[0, 4, 4, 4, 4, 4, 0]
[3, 0, 3, 3, 3, 3, 0]
[4, 2, 0, 4, 5, 5, 0]
[1, 3, 3, 0, 3, 6, 0]
[3, 3, 3, 3, 0, 6, 0]
[4, 5, 5, 4, 5, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
1->1: []
1->2: [1, 4, 3, 2]
1->3: [1, 4, 3]
1->4: [1, 4]
1->5: [1, 4, 3, 5]
1->6: [1, 4, 6]
1->7: []
2->1: [2, 3, 4, 1]
2->2: []
2->3: [2, 3]
2->4: [2, 3, 4]
2->5: [2, 3, 5]
2->6: [2, 3, 5, 6]
2->7: []
3->1: [3, 4, 1]
3->2: [3, 2]
3->3: []
3->4: [3, 4]
3->5: [3, 5]
3->6: [3, 5, 6]
3->7: []
4->1: [4, 1]
4->2: [4, 3, 2]
4->3: [4, 3]
4->4: []
4->5: [4, 3, 5]
4->6: [4, 6]
4->7: []
5->1: [5, 3, 4, 1]
5->2: [5, 3, 2]
5->3: [5, 3]
5->4: [5, 3, 4]
5->5: []
5->6: [5, 6]
5->7: []
6->1: [6, 4, 1]
6->2: [6, 5, 3, 2]
6->3: [6, 5, 3]
6->4: [6, 4]
6->5: [6, 5]
6->6: []
6->7: []
7->1: []
7->2: []
7->3: []
7->4: []
7->5: []
7->6: []
7->7: []


![Test Case 3](TestCase3.png)

In [5]:
'''
Test Case3: 如上图, 出处:大三上<算法设计与分析>PPT
'''
vertexNum=5
edges=[(1, 2, 200), (1, 3, 100), (1, 4, 500), (1, 5, 500), (2, 3, 200), 
        (2, 4, 1200), (2, 5, 1000), (3, 4, 200), (3, 5, 600), (4, 5, 100)]
adjM=build_adjM(vertexNum, edges, is_directed=False)

shortest_paths:list[list[list[int]]]=floyd(adjM)
for i in range(vertexNum):
    for j in range(vertexNum):
        print(f'{i+1}->{j+1}: {shortest_paths[i][j]}')

[0, 2, 3, 3, 3]
[1, 0, 3, 3, 3]
[1, 2, 0, 4, 4]
[3, 3, 3, 0, 5]
[4, 4, 4, 4, 0]
1->1: []
1->2: [1, 2]
1->3: [1, 3]
1->4: [1, 3, 4]
1->5: [1, 3, 4, 5]
2->1: [2, 1]
2->2: []
2->3: [2, 3]
2->4: [2, 3, 4]
2->5: [2, 3, 4, 5]
3->1: [3, 1]
3->2: [3, 2]
3->3: []
3->4: [3, 4]
3->5: [3, 4, 5]
4->1: [4, 3, 1]
4->2: [4, 3, 2]
4->3: [4, 3]
4->4: []
4->5: [4, 5]
5->1: [5, 4, 3, 1]
5->2: [5, 4, 3, 2]
5->3: [5, 4, 3]
5->4: [5, 4]
5->5: []
